In [ ]:
import torch
import re
import tqdm

## Part 1: Data

We iterate through the Netflix dataset text files and prepare the matrix M, a matrix with columns of movies and users. If a user has watched a movie, the index corresponding to the (movie, user) pair will be set to 1, else 0.

### Data prep

We do a simple regex to get the movie patterns and then find all the users that have watched that movie.

In [ ]:
movie_expression = re.compile(r"(\d+):")

def parse(lines: list):
    movie_id = None
    movies_and_users = []
    for line in tqdm.tqdm(lines):
        is_movie = movie_expression.search(line)
        if is_movie:
            movie_id = is_movie.groups()[0]
            continue
        user_id, _, _ = line.split(',')
        movies_and_users.append((int(movie_id), int(user_id)))
    return movies_and_users

In [ ]:
files = [
    "dataset/combined_data_3.txt", # uncomment the below line to process the full dataset, but be warned it's huge.
#     "/kaggle/input/netflix-prize-data/combined_data_4.txt", "/kaggle/input/netflix-prize-data/combined_data_1.txt", "/kaggle/input/netflix-prize-data/combined_data_2.txt"
]
movies_and_users = []
for f in files:
    print(f'processing file {f}')
    with open(f, "r") as raw_text:
        lines = raw_text.readlines()
    movies_and_users.extend(parse(lines[:260578])) # even just one file is BIG so I've just been taking a subset of ~250,000 lines, remove the slice to process the full file
    print(f'completed processing file {f}')


In [ ]:
movies, users = zip(*movies_and_users)

In [ ]:
print(movies_and_users[:10])
print(movies[:5])
print(users[:5])

We now have movie/user pairs. However, we need to turn the IDs into indexes. Lets do that below to convert this data into something we can index into a matrix.

In [ ]:
unique_movies = sorted(list(set(movies)))
unique_users = sorted(list(set(users)))

movie_to_idx = {movie: i for i, movie in enumerate(unique_movies)}
user_to_idx = {user: i for i, user in enumerate(unique_users)}

normalised_movies = [movie_to_idx[m] for m in movies]
normalised_users = [user_to_idx[u] for u in users]

normalised_movies_and_users = list(zip(normalised_movies, normalised_users))

Now we need to create the matrix M representing the co-occurrence of a movie and a user.

In [ ]:
M = torch.zeros(len(unique_movies), len(unique_users))
M.shape

Lets populate the matrix.

In [ ]:
for movie, user in normalised_movies_and_users:
    M[movie, user] = 1

# Recommendation

We have generated a matrix M of all films and all viewers, with the value of each index *i,j* corresponding to whether a given user *i* watched film *j*. This matrix is factorised into two smaller matrices U, F which represent embeddings for users and films respectively. We can iteratively hold each matrix constant while we tune the other to better match the results in M.

First, lets create our smaller matrices U and F.

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
embedding_dimension = 200

U = torch.randn(len(unique_users), embedding_dimension, requires_grad=True, device=device)
F = torch.randn(len(unique_movies), embedding_dimension, requires_grad=True, device=device)

In [ ]:
print(F)

We can see that our two matrices now exist to embed films and movies. Furthermore, when we transpose one matrix and multiply with the other, we get a matrix with shape equal to size M, see below.

In [ ]:
M_hat = (F @ U.T)

We can use this to create a loss function between our created matrix M_hat and true matrix M, then backpropagate to the embedding networks.

We follow the Google tutorial [here](https://developers.google.com/machine-learning/recommendation/collaborative/matrix) and perform Weighted Matrix Factorization.

The 'weighted' element here comes from the loss function. Because our co-occurrences are distributed sparsely, our model has the option early to learn most effectively by simply classifying every (movie, user) pair as 0. So, we weight the loss from co-occurrences much more highly than the non-co-occurences.

The way we do this is by first masking out the non-occurrences and finding the difference between M and M_hat. This provides the loss for the co-occurrences. Next we do the inverse, masking out the co-occurrences to get the loss for the non-occurrences. Then we add the losses together in a weighted fashion. Simple, right!

In [ ]:
M = M.to(device)
optim = torch.optim.Adam([U, F], 0.01)
# optim = torch.optim.AdamW([U, F], 0.01)

In [ ]:
def training_loop(display):
    M_hat = F @ U.T
    difference = M - M_hat
    loss_1 = (difference * M) ** 2
    non_observation_mask = 1 - M
    loss_2 = (difference * non_observation_mask) ** 2
    loss = loss_1 + (0.005 * loss_2)
    loss = loss.mean()
    loss.backward()
    optim.step()
    optim.zero_grad()
    if display:
        print(loss.item())

In [ ]:
for i in range(500):
    display = True if i % 100 == 0 else False
    training_loop(display)


The loss is decreasing, which means M_hat is increasingly similar to M. Lets check that out directly below.

In [ ]:
M[:,0]

In [ ]:
(F @ U.T)[:,0]

Sure enough, our positive co-occurrences are near 1, and our negative occurrences nearby 0. We did it! One additional potential improvement we could add is an element-wise sigmoid function to our M_hat to get all our outputs near 1.

In [ ]:
sigmoid = torch.nn.Sigmoid()

def training_loop(display):
    M_hat = sigmoid(F @ U.T)
    difference = M - M_hat
    loss_1 = (difference * M) ** 2
    non_observation_mask = 1 - M
    loss_2 = (difference * non_observation_mask) ** 2
    loss = loss_1 + (0.005 * loss_2)
    loss = loss.mean()
    loss.backward()
    optim.step()
    optim.zero_grad()
    if display:
        print(loss.item())

In [ ]:
for i in range(500):
    display = True if i % 100 == 0 else False
    training_loop(display)


In [ ]:
sigmoid((F @ U.T)[:,0])

Interestingly, this doesn't actually provide us much of a benefit. In fact, it does a little worse than where we were before.

## Appendix

### Optimizer

The choice of optimizer happens to be very important. Try training again but substituting AdamW for the much more simple optimizer, SGD. You'll notice the training happens MUCH slower.

In [ ]:
U = torch.randn(len(unique_users), embedding_dimension, requires_grad=True, device=device)
F = torch.randn(len(unique_movies), embedding_dimension, requires_grad=True, device=device)
optim = torch.optim.SGD([U, F], lr=0.01)

In [ ]:
def training_loop(display):
    M_hat = F @ U.T
    difference = M - M_hat
    loss_1 = (difference * M) ** 2
    non_observation_mask = 1 - M
    loss_2 = (difference * non_observation_mask) ** 2
    loss = loss_1 + (0.005 * loss_2)
    loss = loss.mean()
    loss.backward()
    optim.step()
    optim.zero_grad()
    if display:
        print(loss.item())

In [ ]:
for i in range(500):
    display = True if i % 100 == 0 else False
    training_loop(display)

Interesting, right? Let's dig into this a bit more.

There are a few key differences between SGD and Adam. SGD on its own is effectively gradient descent at its most basic. It **only** takes the derivative of the loss with respect to each parameter and then descends the gradient to try and reach a minimum.

Adam is SGD with a few improvements:

1. Momentum: Adam essentially uses exponentially decaying gradients from previous updates to give the current weight update the context of previous updates. The consequence of this is that if previous updates tell us to adjust a weight in some way, and current updates tell us to adjust a weight in the same direction, the magnitude of the weight update will be increased. The key to this is the intuition of momentum -- previous motion in the weights has some impact on current motion.
2. Adaptive Learning Rates: If certain parameters are only seen rarely in learning, Adam will scale up the gradient update.

We can add momentum to our SGD algorithm. Lets see how much that gets us.

In [ ]:
optim = torch.optim.SGD([U, F], momentum=0.9, lr=0.01)

for i in range(500):
    display = True if i % 100 == 0 else False
    training_loop(display)

That's a large improvement. However, there's still a **big** difference between Adam and SGD. And this makes sense. Our matrix is largely sparse, and embeddings (especially for users) are updated only a few times each epoch. Having adaptive learning rates which learn a lot for those users makes sense. Lets take away the momentum and just look at adaptive learning. 

We can do this using AdaGrad, which solely modifies SGD to magnify rare (or regularly small) parameter updates.

In [ ]:
optim = torch.optim.Adagrad([U, F], lr=0.01)

for i in range(500):
    display = True if i % 100 == 0 else False
    training_loop(display)

This gets us most of the way to Adam's performance! So, adaptive learning weights turn out to be *crucial* for learning in these sorts of factorization problems. Good to know!

Thank you for following along in this tutorial with me. I hope its been some help. Happy optimizing 👋🤖